In [ ]:
import re
from requests import get
from bs4 import BeautifulSoup
import lxml
import html5lib
import pandas as pd
import time
import numpy as np

In [ ]:
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
params ={
    'q': '도시 문제',
    'tbm': 'nws',
    'start': '0'
}
resp = get('https://www.google.com/search', params=params, headers={'user-agent': ua})

In [ ]:
dom = BeautifulSoup(resp.content, 'html.parser')
for tag in dom.select('a.WlydOe'):
    print(tag.attrs['href'])

In [ ]:
def getUrls():
    ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    depth = 0
    
    result = list()

    while(depth < 51):
        
        params = {
            'q': '도시 문제',
            'tbm': 'nws',
            'start': str(depth)
        }   

        resp = get('https://www.google.com/search', params=params, headers={'user-agent': ua})
        dom = BeautifulSoup(resp.content, 'html.parser')
        result.append([tag.attrs['href'] for tag in dom.select('a.WlydOe')])

        depth += 10
        time.sleep(10)  # prevent traffic attack
    
    return result

In [ ]:
test = getUrls()

In [ ]:
available = list()

for doc in test:
    for jor in doc:
        try:
            resp = get(jor, headers={'user-agent':ua})
            available.append(resp)
        except:
            print("Failed", jor)
            pass
        

In [ ]:
for av in available:
    dom = BeautifulSoup(av.text, 'html.parser')
    print(list([tag.attrs['src'] for tag in dom.select('img')]))

In [ ]:
text_result = list()

for av in available:
    dom = BeautifulSoup(av.text, 'html.parser')
    text_result.append(re.sub(r'([\n,\t,\r]+?)', '', dom.text))

text_result

In [ ]:
import pickle

with open('result.txt', 'wb') as f:
    pickle.dump(text_result, f)

In [ ]:
# # image
# params = {
#     'q':'도시 문제'
# }
# resp = get('https://www.pinterest.co.kr/search/pins/', params=params)
# node = BeautifulSoup(resp.text, 'html.parser')
# node.select('iframe')

In [ ]:
from konlpy.tag import Kkma

In [ ]:
Kkma().morphs('아버지가방에들어가신다')

In [ ]:
test_morphs = Kkma().morphs(text_result[0])
# 1. morphs

In [ ]:
with open('morphs_result.txt', 'wb') as f:
    pickle.dump(test_morphs, f)
# save

In [ ]:
test_nouns = Kkma().nouns(text_result[0])
# 2. nouns

In [21]:
test_nouns = np.array(test_nouns)
test_morphs = np.array(test_morphs)

In [24]:
test_nouns = np.unique(test_nouns)
test_morphs = np.unique(test_morphs)

In [ ]:
with open('nouns_result.txt', 'wb') as f:
    pickle.dump(test_nouns, f)
# save

In [25]:
with open('unique_nouns_morphs.txt', 'wb') as f:
    pickle.dump((test_morphs, test_nouns), f)